<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_0104fine_tuning_bert_pretrained_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- filename: `2022_0104fine_tuning_bert_pretrained_model.ipynb`
- date; 2022_0104
- source: https://huggingface.co/docs/transformers/master/en/training#preparing-the-datasets

# 訓練済モデルの微調整 PyTorch 版
<!-- # Fine-tuning a pretrained model-->


In [ ]:
# Transformers installation
! pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

# Fine-tuning a pretrained model

このチュートリアルでは，Transformers ライブラリから事前訓練されたモデルを微調整する方法を紹介します。
TensorFlow では Keras と`fit` メソッドを使ってモデルを直接訓練することができます。
PyTorch では，汎用的な訓練ループがないため，🤗 Transformer sライブラリでは [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) というクラスで API を提供しており，モデルの微調整や訓練を簡単に行うことができます。 
次に PyTorch で訓練ループ全体を記述する方法を紹介します。
<!-- In this tutorial, we will show you how to fine-tune a pretrained model from the Transformers library. 
In TensorFlow, models can be directly trained using Keras and the `fit` method. 
In PyTorch, there is no generic training loop so the 🤗 Transformers library provides an API with the class [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) to let you fine-tune or train a model from scratch easily. 
Then we will show you how to alternatively write the whole training loop in PyTorch. -->


モデルを微調整する前に，データセットが必要です。
このチュートリアルでは [IMDB dataset](https://www.imdb.com/interfaces/) で BERT を微調整する方法を紹介します。
課題は，映画のレビューが肯定的か否定的かを分類することです。
他の課題例については [additional-resources](#additional-resources) 節を参照してください。
<!-- Before we can fine-tune a model, we need a dataset. 
In this tutorial, we will show you how to fine-tune BERT on the [IMDB dataset](https://www.imdb.com/interfaces/): the task is to classify whether movie reviews are positive or negative. 
For examples of other tasks, refer to the [additional-resources](#additional-resources) section! -->

<a id='data-processing'></a>

## データセットの準備 <!--## Preparing the datasets-->

In [ ]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/_BZearw7f0w?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

ここでは [🤗 Datasets](https://github.com/huggingface/datasets/) ライブラリを使って，IMDB データセットをダウンロードし，前処理を行います。
この部分は簡単に説明します。
このチュートリアルの焦点は訓練なので，詳細は🤗 Datasets [documentation](https://huggingface.co/docs/datasets/) や [preprocessing](https://huggingface.co/docs/transformers/master/en/preprocessing) のチュートリアルを参照してください。
<!-- We will use the [🤗 Datasets](https://github.com/huggingface/datasets/) library to download and preprocess the IMDB datasets. 
We will go over this part pretty quickly. Since the focus of this tutorial is on training, you should refer to the 🤗 Datasets [documentation](https://huggingface.co/docs/datasets/) or the [preprocessing](https://huggingface.co/docs/transformers/master/en/preprocessing) tutorial for more information. -->


まず `load_dataset` 関数を使って、データセットをダウンロードしてキャッシュします。
<!-- First, we can use the `load_dataset` function to download and cache the dataset: -->

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("imdb")

これは，モデルやトークナイザで見た `from_pretrained` メソッドと同じように動作します  (ただし，キャッシュディレクトリはデフォルトで  _~/.cache/huggingface/dataset_  になっています)。
<!-- This works like the `from_pretrained` method we saw for the models and tokenizers (except the cache directory is _~/.cache/huggingface/dataset_ by default). -->

`raw_datasets` オブジェクトは，3 つのキーを持つ辞書です。
`"train"`, `"test"`, `"unsupervised”` の 3 つのキーを持つ辞書です (これはデータセットの 3 つの分割に対応しています)。
`"train”` の分割は訓練に，`"test”` の分割は検証に使用します。
<!-- The `raw_datasets` object is a dictionary with three keys: `"train"`, `"test"` and `"unsupervised"` (which correspond to the three splits of that dataset). 
We will use the `"train"` split for training and the `"test"` split for validation. -->

データを前処理するためには，トークナイザが必要です。
<!-- To preprocess our data, we will need a tokenizer: -->

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

[前処理](https://huggingface.co/docs/transformers/master/en/preprocessing) で見たように，以下のコマンドでモデル用のテキスト入力を準備することができます (これは例であって，実行できるコマンドではありません)。
<!-- As we saw in [preprocessing](https://huggingface.co/docs/transformers/master/en/preprocessing), we can prepare the text inputs for the model with the following command (this is an example, not a command you can execute): -->

In [ ]:
sentences = 'This is a pen.'

In [ ]:
inputs = tokenizer(sentences, padding="max_length", truncation=True)

これにより，すべてのサンプルは，パディングまたは切り詰めによって，モデルが受け入れられる最大の長さ (ここでは512) になります。

しかし，代わりに，`map` メソッドを使って，これらの前処理をデータセットのすべての分割に一度に適用することができます。
<!-- This will make all the samples have the maximum length the model can accept (here 512), either by padding or truncating them.

However, we can instead apply these preprocessing steps to all the splits of our dataset at once by using the `map` method: -->

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

マップ法や，データを前処理する他の方法については，🤗 Datasets [document](https://huggingface.co/docs/datasets/) で詳しく説明しています。

次に，訓練セットと検証セットの小さなサブセットを生成して，訓練の高速化を図ります。
<!-- You can learn more about the map method or the other ways to preprocess the data in the 🤗 Datasets [documentation](https://huggingface.co/docs/datasets/).

Next we will generate a small subset of the training and validation set, to enable faster training: -->

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

以下のすべての例では，常に `small_train_dataset` と `small_eval_dataset` を使用しています。
完全なデータセットで訓練や評価を行うには，これらを _full_ に置き換えるだけです。
<!-- In all the examples below, we will always use `small_train_dataset` and `small_eval_dataset`. Just replace them by their _full_ equivalent to train or evaluate on the full dataset. -->

<a id='trainer'></a>

## Trainer API による PyTorch の微調整
<!-- ## Fine-tuning in PyTorch with the Trainer API -->

In [ ]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/nvBXf7s7vTI?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

PyTorch は訓練ループを提供していないので 🤗 Transformers ライブラリは，🤗 Transformers モデルに最適化された [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) という API を提供しており，幅広い訓練オプションと，ロギング，勾配累積，混合精度などの機能が組み込まれています。

まず，モデルを定義しましょう。
<!-- Since PyTorch does not provide a training loop, the 🤗 Transformers library provides a [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) API that is optimized for 🤗 Transformers models, with a wide range of training options and with built-in features like logging, gradient accumulation, and mixed precision.

First, let's define our model: -->

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

これにより，事前学習された重みの一部が使用されず，一部の重みがランダムに初期化されるという警告が表示されます。
これは，BERT モデルの事前学習ヘッドを破棄して，ランダムに初期化された分類ヘッドに置き換えているためです。
我々は，事前学習されたモデルの知識を移行しながら，課題上でこのモデルを微調整します (これが，転移学習と呼ばれる理由です)。
<!-- This will issue a warning about some of the pretrained weights not being used and some weights being randomly initialized. 
That's because we are throwing away the pretraining head of the BERT model to replace it with a classification head which is randomly initialized. 
We will fine-tune this model on our task, transferring the knowledge of the pretrained model to it (which is why doing this is called transfer learning).-->

次に [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer)を定義するために， [TrainingArguments](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.TrainingArguments) をインスタンス化する必要があります。
このクラスには，[Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) で調整できるすべてのハイパーパラメータや，サポートしているさまざまな学習オプションを有効にするフラグが含まれています。
チェックポイントを保存するためのディレクトリを指定するだけです。
<!-- Then, to define our [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer), we will need to instantiate a [TrainingArguments](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.TrainingArguments). 
This class contains all the hyperparameters we can tune for the [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) or the flags to activate the different training options it supports. 
Let's begin by using all the defaults, the only thing we then have to provide is a directory in which the checkpoints will be saved: -->

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")

それでは，[Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) を次のように実体化しましょう。
<!-- Then we can instantiate a [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) like this: -->

In [ ]:
from transformers import Trainer

trainer = Trainer(model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset)

モデルを微調整するためには，ただ呼び出すだけです。
<!-- To fine-tune our model, we just need to call -->

In [ ]:
trainer.train()

を実行すると，プログレスバーで確認できる訓練が開始され，数分で完了します (GPU にアクセスできる限り)。
デフォルトでは，訓練中に評価は行われず，[Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) に指標を計算するように指示していないので，モデルの性能がどの程度かについては，実際には何もわかりません。
では，その方法を見てみましょう。
<!-- which will start a training that you can follow with a progress bar, which should take a couple of minutes to complete (as long as you have access to a GPU). 
It won't actually tell you anything useful about how well (or badly) your model is performing however as by default, there is no evaluation during training, and we didn't tell the [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) to compute any metrics. Let's have a look on how to do that now!-->

[Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) がメトリクスを計算して報告するには，予測値とラベル（[EvalPrediction](https://huggingface.co/docs/transformers/master/en/internal/trainer_utils#transformers.EvalPrediction) という名前のついたタプルにまとめられている) を受け取り，文字列アイテム (メトリクス名) と浮動小数点値 (メトリクス値) を含む辞書を返す `compute_metrics` 関数を与える必要があります。
<!-- To have the [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) compute and report metrics, we need to give it a `compute_metrics` function that takes predictions and labels (grouped in a namedtuple called [EvalPrediction](https://huggingface.co/docs/transformers/master/en/internal/trainer_utils#transformers.EvalPrediction)) and return a dictionary with string items (the metric names) and float values (the metric values). -->

🤗 Datasets ライブラリでは，NLP  で使われる一般的なメトリクスを `load_metric` 関数で簡単に取得することができます。
ここでは，単純に `accuracy` を使います。
次に，対数を予測値に変換するだけの `compute_metrics` 関数を定義し (すべての🤗 Transformers モデルは対数を返すことを覚えておいてください)， それをこのメトリクスの  `compute`  メソッドに与えます。
<!-- The 🤗 Datasets library provides an easy way to get the common metrics used in NLP with the `load_metric` function. 
here we simply use accuracy. 
Then we define the `compute_metrics` function that just convert logits to predictions (remember that all 🤗 Transformers models return the logits) and feed them to `compute` method of this metric. -->

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

`compute` 関数は，(ロジットとラベルを含む) タプルを受け取り，文字列キー (メトリック名) と浮動小数点値を含む辞書を返さなければなりません。
この関数は，各評価フェーズの終わりに，予測値やラベルの配列全体に対して呼び出されます。
<!-- The compute function needs to receive a tuple (with logits and labels) and has to return a dictionary with string keys (the name of the metric) and float values. 
It will be called at the end of each evaluation phase on the whole arrays of predictions/labels.-->

実際に動作するかどうかを確認するために，微調整されたモデルで新しい [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) を作成してみましょう。
<!-- To check if this works on practice, let's create a new [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) with our fine-tuned model: -->

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

で 87.5%の精度が得られました。
<!-- which showed an accuracy of 87.5% in our case. -->

モデルを微調整し，評価指標を定期的に (例えば，各エポックの終わりに) 報告したい場合，訓練引数をどのように定義すべきかを示します。
<!-- If you want to fine-tune your model and regularly report the evaluation metrics (for instance at the end of each epoch), here is how you should define your training arguments: -->

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer", evaluation_strategy="epoch")

その他のオプションについては [TrainingArguments](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.TrainingArguments) のドキュメントを参照してください。
<!-- See the documentation of [TrainingArguments](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.TrainingArguments) for more options. -->


<a id='keras'></a>

## Keras による微調整
<!-- ## Fine-tuning with Keras -->

In [ ]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/rnTGBy2ax1c?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

モデルは，Keras API を使って TensorFlow でネイティブに訓練することもできます。まず，モデルを定義しましょう。
<!-- Models can also be trained natively in TensorFlow using the Keras API. First, let's define our model: -->

In [ ]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

次に，先ほどのデータセットを標準の `tf.data.Dataset` に変換する必要があります。
Shape が固定されているので，以下のように簡単に行うことができます。
まず，データセットから _"text”_ 列を削除し，TensorFlow フォーマットに設定します。
<!-- Then we will need to convert our datasets from before in standard `tf.data.Dataset`. 
Since we have fixed shapes, it can easily be done like this. First we remove the _"text"_ column from our datasets and set them in TensorFlow format: -->

In [ ]:
tf_train_dataset = small_train_dataset.remove_columns(["text"]).with_format("tensorflow")
tf_eval_dataset = small_eval_dataset.remove_columns(["text"]).with_format("tensorflow")

そして，すべてを大きなテンソルに変換し，`tf.data.Dataset.from_tensor_slices` メソッドを使用します。
<!-- Then we convert everything in big tensors and use the `tf.data.Dataset.from_tensor_slices` method: -->

In [ ]:
train_features = {x: tf_train_dataset[x] for x in tokenizer.model_input_names}
train_tf_dataset = tf.data.Dataset.from_tensor_slices((train_features, tf_train_dataset["label"]))
train_tf_dataset = train_tf_dataset.shuffle(len(tf_train_dataset)).batch(8)

eval_features = {x: tf_eval_dataset[x] for x in tokenizer.model_input_names}
eval_tf_dataset = tf.data.Dataset.from_tensor_slices((eval_features, tf_eval_dataset["label"]))
eval_tf_dataset = eval_tf_dataset.batch(8)

このようにして作成されたモデルは，他の Keras モデルと同様にコンパイルして学習することができます。
<!-- With this done, the model can then be compiled and trained as any Keras model: -->

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

model.fit(train_tf_dataset, validation_data=eval_tf_dataset, epochs=3)

TensorFlow モデルと PyTorch モデルの間の緊密な相互運用性により，モデルを保存してから PyTorch モデルとして再読み込みすることもできます (またはその逆)。
<!-- With the tight interoperability between TensorFlow and PyTorch models, you can even save the model and then reload it as a PyTorch model (or vice-versa): -->

In [ ]:
from transformers import AutoModelForSequenceClassification

model.save_pretrained("my_imdb_model")
pytorch_model = AutoModelForSequenceClassification.from_pretrained("my_imdb_model", from_tf=True)

<a id='pytorch_native'></a>

## 生の PyTorch を使った微調整
<!-- ## Fine-tuning in native PyTorch -->

In [ ]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/Dh9CL8fyG80?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

この段階でノートブックを再起動してメモリを解放するか，次のコードを実行する必要があるかもしれません。
<!-- You might need to restart your notebook at this stage to free some memory, or execute the following code: -->

In [ ]:
del model
del pytorch_model
del trainer
torch.cuda.empty_cache()

NameError: ignored

それでは [trainer section](#trainer) と同じ結果を PyTorch で実現する方法を見てみましょう。
まず最初に，バッチを反復処理するためのデータローダを定義する必要があります。
その前に `tokenized_datasets` にちょっとした後処理を施す必要があります。
<!-- Let's now see how to achieve the same results as in [trainer section](#trainer) in PyTorch. 
First we need to define the dataloaders, which we will use to iterate over batches. We just need to apply a bit of post-processing to our `tokenized_datasets` before doing that to: -->

- モデルが想定していない値に対応するカラムを削除する (ここでは `"text"` カラム)。
- “label" カラムの名前を `"labels"` に変更する (モデルは引数に `labels` という名前がつくことを想定しているため)。
- データセットのフォーマットを設定し，リストではなく PyTorch テンソルを返すようにします。

<!-- - remove the columns corresponding to values the model does not expect (here the `"text"` column)
- rename the column `"label"` to `"labels"` (because the model expect the argument to be named `labels`)
- set the format of the datasets so they return PyTorch Tensors instead of lists. -->


我々の _tokenized_datasets_ は，これらのステップごとに 1 つのメソッドを持っています。
<!-- Our _tokenized_datasets_ has one method for each of those steps: -->

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

これで，データローダを簡単に定義できるようになりました。
<!-- Now that this is done, we can easily define our dataloaders: -->

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

次に，モデルを定義します：
<!-- Next, we define our model: -->

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file https://huggingface.co/bert-base-cased/resolve/

あとは，最適化と学習率スケジューラの 2 つだけです。
[Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) で使われているデフォルトの最適化は [AdamW](https://huggingface.co/docs/transformers/master/en/main_classes/optimizer_schedules#transformers.AdamW) です。
<!-- We are almost ready to write our training loop, the only two things are missing are an optimizer and a learning rate scheduler. 
The default optimizer used by the [Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer) is [AdamW](https://huggingface.co/docs/transformers/master/en/main_classes/optimizer_schedules#transformers.AdamW): -->

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

最後に，デフォルトで使用されている学習率スケジューラは，最大値 (ここでは5e-5) から 0 までの直線的な減衰だけです。
<!-- Finally, the learning rate scheduler used by default is just a linear decay from the maximum value (5e-5 here) to 0: -->

In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

最後に，GPU が利用できる場合は，GPU を使用したいと思います (そうしないと，訓練が数分ではなく，数時間かかる可能性があります)。
そのためには，モデルとバッチを配置する「デバイス」を定義します。
<!-- One last thing, we will want to use the GPU if we have access to one (otherwise training might take several hours instead of a couple of minutes). 
To do this, we define a `device` we will put our model and our batches on. -->

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

In [ ]:
#device

これで訓練の準備が整いました。
訓練がいつ終了するかを知るために _tqdm_ ライブラリを使って，訓練のステップ数を示すプログレスバーを追加しました。
<!-- We now are ready to train! To get some sense of when it will be finished, we add a progress bar over our number of training steps, using the _tqdm_ library. -->

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/375 [00:00<?, ?it/s]

もしあなたが (コンピュータビジョンのように) 事前学習したモデルのボディを凍結させることに慣れているのであれば，何の予防措置も取らずにモデル全体を直接微調整しているため，上記は少し奇妙に見えるかもしれないことに注意してください。
トランスフォーマーモデルでは，実際にこの方法でうまく機能しています (なので，これは私たち側の見落としではありません)。
モデルの「ボディの凍結」が何を意味するのかわからない方は，この段落を読むのを忘れてください。
<!-- Note that if you are used to freezing the body of your pretrained model (like in computer vision) the above may seem a bit strange, as we are directly fine-tuning the whole model without taking any precaution. 
It actually works better this way for Transformers model (so this is not an oversight on our side). 
If you're not familiar with what "freezing the body" of the model means, forget you read this paragraph.-->

さて，結果を確認するためには，評価ループを書かなければなりません。
[トレーナーセクション](#trainer) のように，データセットライブラリの指標を使います。
ここでは，各バッチごとに予測値を蓄積し，ループが終了したときに最終結果を計算します。
<!--Now to check the results, we need to write the evaluation loop. Like in the [trainer section](#trainer) we will use a metric from the datasets library. 
Here we accumulate the predictions at each batch before computing the final result when the loop is finished. -->


In [ ]:
metric = load_metric("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

<a id='additional-resources'></a>

## 追加資源
<!-- ## Additional resources -->

より詳細な調整例を見るには，以下を参照してください。
<!-- To look at more fine-tuning examples you can refer to:-->

- [🤗 Transformers Examples](https://github.com/huggingface/transformers/tree/master/examples) には PyTorch とTensorFlow で一般的な NLP 課題を学習するスクリプトが含まれています。

- [🤗 Transformers Notebooks](https://huggingface.co/docs/transformers/master/en/notebooks) には様々なノートブックが含まれており，特に課題ごとのノートブックがあります (_how to finetune a model on xxx_ を探してみてください)。

<!-- 
- [🤗 Transformers Examples](https://github.com/huggingface/transformers/tree/master/examples) which includes scripts to train on all common NLP tasks in PyTorch and TensorFlow.

- [🤗 Transformers Notebooks](https://huggingface.co/docs/transformers/master/en/notebooks) which contains various notebooks and in particular one per task (look for the _how to finetune a model on xxx_). -->